# Topic Modeling

Topic modelling can be described as a method for finding a group of words (i.e topic) from a collection of documents that best represents the information in the collection. It can also be thought of as a form of text mining – a way to obtain recurring patterns of words in textual material.
In this project I will use unsupervised techniques, to cluster/ group reviews to identify main topics/ ideas in the sea of text. This will be applicable to any textual reviews but I will focus on twitter data which is more real world and more complex than reviews obtained from review or survey forms. 


In [ ]:
import nltk
import numpy as np
import re # remove regex
import pandas as pd 

%matplotlib inline



In [ ]:
## Get multiple outputs in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
## Display all rows and columns of a dataframe instead of a truncated version
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load data

In [ ]:
df  = pd.read_csv("tweets.csv",encoding = 'ISO-8859-1')
df.head()

In [ ]:
unique_text = df.tweet.unique()
print(len(unique_text))

In [ ]:
df['tweet'][123]

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt


### Remove any @ mentions

In [ ]:
df['Clean_text'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*")
df['Clean_text']

In [ ]:
df['Clean_text'] = df['Clean_text'].str.replace("[^a-zA-Z#]", " ")
df['Clean_text']

In [ ]:
df["Clean_text"]= df["Clean_text"].str.lower() 
df['Clean_text']

In [ ]:
df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df.head()

In [ ]:
tokenized_tweet = df['Clean_text'].apply(lambda x: x.split())
tokenized_tweet.head()

In [ ]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df['Clean_text'] = tokenized_tweet

In [ ]:
df

In [ ]:
df.loc[:,('Clean_text')]

In [ ]:
df.drop_duplicates(subset=['Clean_text'], keep = 'first',inplace= True)


In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df

In [ ]:
df['Clean_text_length'] = df['Clean_text'].apply(len)
df.head()

In [ ]:
df[df['Clean_text_length']==0]

In [ ]:
df[df['Clean_text_length']==0]['Clean_text'] ## Looks like these are tweets with different languages or just hastags.
# We can simply drop these tweets
indexes = df[df['Clean_text_length']==0]['Clean_text'].index
indexes

In [ ]:
df.drop(index = indexes,inplace=True)

In [ ]:
df.info()

In [ ]:
df.reset_index(drop=True,inplace=True)
df.info()

In [ ]:
df['Clean_text'].head()

## Vectorizer

CountVectorizer:

Convert a collection of text documents to a matrix of token counts
The CountVectorizer provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary\
Number of features will be equal to the vocabulary size found by analyzing the data.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer='word',ngram_range=(1,1), stop_words='english', min_df = 0.0001, max_df=0.7)
count_vect.fit(df['Clean_text'])
desc_matrix = count_vect.transform(df["Clean_text"])
desc_matrix

In [ ]:
desc_matrix.toarray()

In [ ]:
desc_matrix.shape

## Clustering

In [ ]:
from sklearn.cluster import KMeans
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import matplotlib.pyplot as plt

In [ ]:
def wordcloud(cluster):
  # combining the image with the dataset
  Mask = np.array(Image.open(requests.get('http://clipart-library.com/image_gallery2/Twitter-PNG-Image.png', stream=True).raw))

  # We use the ImageColorGenerator library from Wordcloud 
  # Here we take the color of the image and impose it over our wordcloud
  image_colors = ImageColorGenerator(Mask)

  # Now we use the WordCloud function from the wordcloud library 
  wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask).generate(cluster)

  # Size of the image generated 
  plt.figure(figsize=(10,20))

  # Here we recolor the words from the dataset to the image's color
  # recolor just recolors the default colors to the image's blue color
  # interpolation is used to smooth the image generated 
  plt.imshow(wc.recolor(color_func=image_colors),interpolation="hamming")

  plt.axis('off')
  plt.show()

In [ ]:
def identify_topics(df, desc_matrix, num_clusters):
    km = KMeans(n_clusters=num_clusters)
    km.fit(desc_matrix)
    clusters = km.labels_.tolist()
    tweets = {'Tweet': df["Clean_text"].tolist(), 'Cluster': clusters}
    frame = pd.DataFrame(tweets, index = [clusters])
    print(frame['Cluster'].value_counts())

    
    for cluster in range(num_clusters):
        cluster_words = ' '.join(text for text in frame[frame['Cluster'] == cluster]['Tweet'])
        wordcloud(cluster_words)
        


In [ ]:
identify_topics(df, desc_matrix, 2)

In [ ]:
identify_topics(df, desc_matrix, 6)

6 seems like a good number of clusters

In [ ]:
frame.to_csv('clustered_tweets.csv') 